# **<font color='green'> Quantum State Reconstruction for pure three-qubit states</font>**
Here, we propose our neural network-based quantum state reconstruction method for pure three-qubit states.

# **<font color='green'> Libraries and Random Seeds</font>**

In [ ]:
!pip install qiskit
!pip install qiskit-aer

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.linalg
import qiskit as qk
import qiskit.visualization

from qiskit.circuit import ParameterVector
from qiskit.quantum_info import Statevector

from qiskit_aer import AerSimulator
from qiskit_aer.noise import QuantumError, ReadoutError, NoiseModel,depolarizing_error,amplitude_damping_error,kraus_error,pauli_error
from qiskit import transpile
from qiskit.quantum_info.operators import Operator

import tensorflow as tf
from tensorflow.python.framework.ops import convert_to_tensor
print("TensorFlow version:", tf.__version__)

import keras
from keras.layers import Dense, Input, Lambda, concatenate, Conv2D, UpSampling2D, MaxPooling2D, Add,Concatenate,Flatten, Conv1D, Conv1DTranspose, UpSampling1D,MaxPooling1D, BatchNormalization
from tensorflow.keras import Model

TensorFlow version: 2.17.0


In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService



In [ ]:
# Ensure reproducibility
np.random.seed(123)
tf.random.set_seed(123)
import random
random.seed(10)

# **<font color='green'> Functions</font>**

In [ ]:
# Generation of a random unitary transformation
def random_unitary(N):
    Z=np.random.randn(N,N) + 1.0j * np.random.randn(N,N)
    [Q,R]=sp.linalg.qr(Z)
    D=np.diag(np.diagonal(R)/np.abs(np.diagonal(R)))
    return np.dot(Q,D)

In [ ]:
# Generate the density matrix given the state vector
def get_density_matrix(state_vector):
    density_matrix = np.outer(state_vector, np.conjugate(state_vector))
    return density_matrix

In [ ]:
# Here we define the identity matrix and the Pauli matrices for dimension 2 (one qubit)
I = np.array([[1, 0],[0, 1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

In [ ]:
sim_bknd=AerSimulator()

In [ ]:
# Generate Haar distributed data

def generate_Haar_data(num_qubits, samples=1000):
    data = []
    for i in range(samples):
        qc = qk.QuantumCircuit(num_qubits) #creates a quantum circuit with "num_qubits" qubits
        u = random_unitary(2**num_qubits)
        qc.unitary(u, qubits=range(num_qubits)) #applies the random unitary transformation to the circuit
        qc = qk.transpile(qc, backend=sim_bknd) #it's used to optimize the circuit
        qc.save_statevector() #it's the instruction to save the state vector obtained by the simulation
        state = sim_bknd.run(qc).result().get_statevector(qc) #does the simulation and gets the state vector
        state = np.asarray(state)
        data.append(state)
    return data

In [ ]:
# Generate Pauli basis for three-qubit states

pauli_basis1q = np.array([I, X, Y, Z])
pauli_basis2q = np.array([np.kron(a,b) for a in pauli_basis1q for b in pauli_basis1q])

pauli_basis3q = np.array([np.kron(a,b) for a in pauli_basis2q for b in pauli_basis1q])
pauli_basis3q_modified = pauli_basis3q[1:]


# Compute Bloch components from density matrix
def bloch_coeffs(rho):
    c = []
    for p in pauli_basis3q_modified:
        c.append(np.trace(rho @ p).real)
    return np.array(c)

In [ ]:
# 8x8 Identity matrix
I_8 = tf.eye(8, dtype=tf.complex64)

# Compute fidelity between generalized Bloch vectors
def fid_threeq(a,b):
   a = tf.cast(a, tf.complex64)
   b = tf.cast(b, tf.complex64)
   el_a = tf.einsum('ijk,mi->mjk', pauli_basis3q_modified, a)
   el_b = tf.einsum('ijk,mi->mjk', pauli_basis3q_modified, b)
   rho_a = 0.125 *(el_a + I_8)
   rho_b = 0.125 * (el_b +I_8)
   fidelity = tf.linalg.trace(rho_a @ rho_b)
   return fidelity

In [ ]:
# Define Infidelity as loss function
@tf.function
def infidelity3(a,b):
   a = tf.cast(a, tf.complex64)
   b = tf.cast(b, tf.complex64)
   el_a = tf.einsum('ijk,mi->mjk', pauli_basis3q_modified, a)
   el_b = tf.einsum('ijk,mi->mjk', pauli_basis3q_modified, b)
   rho_a = 0.125 *(el_a + I_8)
   rho_b = 0.125 * (el_b +I_8)
   fidelity = tf.linalg.trace(rho_a @ rho_b)
   infidelity = 1 - fidelity
   infidelity = tf.cast(infidelity, dtype = tf.float32)
   return infidelity

# **<font color='green'> State Reconstruction</font>**

In [ ]:
# Generate data
data = generate_Haar_data(3, 4000)
density_matrix_noise_free = [*map(get_density_matrix, data)]

In [ ]:
service = QiskitRuntimeService(channel='ibm_quantum',token='APIKEYREMOVED')
backend = service.backend("ibm_brisbane")

In [ ]:
noise_model = NoiseModel.from_backend(backend)
noisy_bknd = AerSimulator(method = 'density_matrix', noise_model=noise_model)

In [ ]:
num_qubits = 3
den_mat_noise_free = [] # noise-free density matrices
den_mat_with_noise = [] # noisy density matrices


for i in range(len(data)):
    #print(f"Sample: {i+1} / {len(data)}")

    circ = qk.QuantumCircuit(num_qubits)

    circ.initialize(data[i])
    circ = transpile(circ, backend=AerSimulator(method='density_matrix'))
    circ.barrier()
    circ.id(0) # applies identity to the first qubit
    circ.id(1) # applies identity to the second qubit
    circ.id(2) # applies identity to the third qubit
    circ.save_density_matrix()
    #print(circ)
    ideal_bknd = AerSimulator()
    res = ideal_bknd.run(circ).result().data()['density_matrix'] # simulate the circuit and save the density matrix
    den_noise_free = np.asarray(res)
    den_mat_noise_free.append(den_noise_free)


    res_noise = noisy_bknd.run(circ).result().data()['density_matrix']
    den_with_noise = np.asarray(res_noise)
    den_mat_with_noise.append(den_with_noise)

In [ ]:
bloch_vectors_with_noise = [*map(bloch_coeffs, den_mat_with_noise)] # noise-free Bloch vectors
bloch_vectors_noise_free = [*map(bloch_coeffs, den_mat_noise_free)] # noisy Bloch vectors

In [ ]:
# Generate training, validation and test set

x_train_list = bloch_vectors_with_noise[:900]
y_train_list = bloch_vectors_noise_free[:900]

x_train = tf.convert_to_tensor(x_train_list)
y_train = tf.convert_to_tensor(y_train_list)

x_val_list = bloch_vectors_with_noise[1000:1500]
y_val_list = bloch_vectors_noise_free[1000:1500]

x_val = tf.convert_to_tensor(x_val_list)
y_val = tf.convert_to_tensor(y_val_list)

x_test_list = bloch_vectors_with_noise[1500:]
y_test_list = bloch_vectors_noise_free[1500:]

x_test = tf.convert_to_tensor(x_test_list)
y_test = tf.convert_to_tensor(y_test_list)

## **<font color='teal'> Training with MSE</font>**

mlp model

In [ ]:
# Define Model
mlpmodel = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(63,)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(63),
  tf.keras.layers.Lambda(lambda x:  tf.math.sqrt(7.0) * tf.math.l2_normalize(x, axis=1))
  ])
# Compile model
loss_fn = tf.keras.losses.mse
adam_opt = tf.optimizers.Adam(0.0002)
mlpmodel.compile(optimizer=adam_opt,
              loss=loss_fn)
mlpmodel.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           8,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 63)                  │           8,127 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda (Lambda)                      │ (None, 63)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 49,343 (192.75 KB)

 Trainable params: 49,343 (192.75 KB)

 Non-trainable params: 0 (0.00 B)

mlp training

In [ ]:
# Train Model
history = mlpmodel.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=100, epochs=800)


Epoch 1/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - loss: 0.2237 - val_loss: 0.2183
Epoch 2/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2159 - val_loss: 0.2128
Epoch 3/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2087 - val_loss: 0.2065
Epoch 4/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2005 - val_loss: 0.1990
Epoch 5/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1907 - val_loss: 0.1898
Epoch 6/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1793 - val_loss: 0.1792
Epoch 7/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1671 - val_loss: 0.1686
Epoch 8/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1555 - val_loss: 0.1592
Epoch 9/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1454 - val_loss: 0.1512
Epoch 10/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1365 - val_loss: 0.1439
Epoch 11/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1288 - val_loss: 0.1374
Epoch 12/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1218 - val_loss: 0.1315


mlp eval

In [ ]:
score = mlpmodel.evaluate(x_test,  y_test, verbose=2)

7/7 - 0s - 20ms/step - loss: 4.1167e-04


In [ ]:
# save the model predictions in a tensor
y_prediction = mlpmodel(x_test)

y_prediction = y_prediction.numpy().tolist()

y_test = tf.cast(y_test, dtype=tf.float32)
#x_test = tf.cast(x_test, dtype=tf.float32)


fidelities = fid_threeq(y_prediction, y_test)


print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(fidelities).numpy()}")

Fidelity between ideal and predicted Bloch vectors: (0.9983790516853333+1.4988472962773614e-10j)


## **<font color='teal'> Training with Infidelity</font>**

In [ ]:
# Define Model
model2 = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(63,)),
  tf.keras.layers.Dense(128, activation='relu'),
   tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(63),
  tf.keras.layers.Lambda(lambda x:  tf.math.sqrt(7.0) * tf.math.l2_normalize(x, axis=1))
  ])

# Compile model
adam_opt = tf.optimizers.Adam(0.0002)
model2.compile(optimizer=adam_opt,
              loss=infidelity3)
model2.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 128)                 │           8,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 63)                  │           8,127 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_2 (Lambda)                    │ (None, 63)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 49,343 (192.75 KB)

 Trainable params: 49,343 (192.75 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train Model
history = model2.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=100, epochs=800)

Epoch 1/800


9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 0.8471 - val_loss: 0.8297
Epoch 2/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.8100 - val_loss: 0.8016
Epoch 3/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.7741 - val_loss: 0.7690
Epoch 4/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.7346 - val_loss: 0.7315
Epoch 5/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.6914 - val_loss: 0.6918
Epoch 6/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.6476 - val_loss: 0.6537
Epoch 7/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.6068 - val_loss: 0.6200
Epoch 8/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.5705 - val_loss: 0.5907
Epoch 9/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.5385 - val_loss: 0.5645
Epoch 10/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.5098 - val_loss: 0.5407
Epoch 11/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.4840 - val_loss: 0.5188
Epoch 12/800
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.4606 - val_loss: 0.4987
Epoch 

In [ ]:
score = model2.evaluate(x_test,  y_test, verbose=2)

7/7 - 0s - 29ms/step - loss: 0.0013


In [ ]:
# save the model predictions in a tensor
y_prediction = model2(x_test)
y_prediction = y_prediction.numpy().tolist()

y_test = tf.cast(y_test, dtype=tf.float32)
#x_test = tf.cast(x_test, dtype=tf.float32)


fidelities = fid_threeq(y_prediction, y_test)


print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(fidelities).numpy()}")

Fidelity between ideal and predicted Bloch vectors: (0.9986683130264282+1.0375515724359019e-10j)
